In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import re

In [ ]:
df = pd.read_csv("data/IMDB_Dataset.csv")
mask = (df['review'].str.len() < 400)
df = df.loc[mask]
df['review'] = df['review'].fillna('')
features = df["review"]
labels = df["sentiment"]
labels = np.where(labels == "positive", 1, 0)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)

In [ ]:
def max_len_vocab(data):
    max_len = 0
    vocab = set()
    for i in data:
        if type(i) != type(''):
            continue
        if len(i) > max_len:
            max_len = len(i)
        alpha = re.findall(r'[A-Za-z]*', i.lower())
        for j in alpha:
            vocab.add(j)
    vocab.remove('')
    return max_len, list(vocab)

max_len, vocab = max_len_vocab(features)

In [ ]:
def Review_model():
    reviews = keras.layers.Input(shape=(1,), dtype="string")
    vectorization = keras.layers.TextVectorization(max_tokens=len(vocab)+2, output_mode='int',
                                         output_sequence_length=max_len, vocabulary=vocab)(reviews)
    embedding_layer = keras.layers.Embedding(len(vocab)+2, 10)(vectorization)
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128))(embedding_layer)
    classifier = keras.layers.Dense(32, activation='relu')(lstm) 
    output = keras.layers.Dense(1, activation='sigmoid')(classifier) 
    model = keras.Model(inputs=reviews, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), metrics='accuracy')
    return model

In [ ]:
model = Review_model()
model.fit(X_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
32/32 [==============================] - 159s 5s/step - loss: 0.6859 - accuracy: 0.5496
Epoch 2/10
32/32 [==============================] - 196s 6s/step - loss: 0.7499 - accuracy: 0.5919
Epoch 3/10
32/32 [==============================] - 242s 8s/step - loss: 0.6349 - accuracy: 0.7240
Epoch 4/10
32/32 [==============================] - 216s 7s/step - loss: 0.5339 - accuracy: 0.7952
Epoch 5/10
32/32 [==============================] - 201s 6s/step - loss: 0.3990 - accuracy: 0.8670
Epoch 6/10
32/32 [==============================] - 174s 5s/step - loss: 0.2623 - accuracy: 0.9238
Epoch 7/10
32/32 [==============================] - 170s 5s/step - loss: 0.1673 - accuracy: 0.9487
Epoch 8/10
32/32 [==============================] - 148s 5s/step - loss: 0.0902 - accuracy: 0.9821
Epoch 9/10
32/32 [==============================] - 145s 5s/step - loss: 0.0449 - accuracy: 0.9920
Epoch 10/10
32/32 [==============================] - 165s 5s/step - loss: 0.0225 - accuracy: 0.9985


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
acc

21/21 [==============================] - 6s 289ms/step - loss: 1.1809 - accuracy: 0.6179


0.6179104447364807

In [ ]:
model.save('IMDB_model')

INFO:tensorflow:Assets written to: IMDB_model\assets


INFO:tensorflow:Assets written to: IMDB_model\assets
